# HELCOM (With callbacks)
> Data pipeline (handler) to convert HELCOM data ([source](https://helcom.fi/about-us)) to `NetCDF` format

The data is provided as a Microsoft Access database. `Mdbtools` (https://github.com/mdbtools/mdbtools) is used to convert tables into `.csv` files on Unix-like OS.

## Packages import

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import fastcore.all as fc
from operator import attrgetter

from netCDF4 import Dataset
from datetime import datetime, timedelta
from cftime import num2date, date2num
from pathlib import Path
from datetime import datetime
import re

from marisco.utils import has_valid_varname, match_worms
from marisco.serializers import to_netcdf
from marisco.configs import get_nc_tpl_path, get_cfgs
from marisco.metadata import Metadata, ZoteroItem

NC_TPL_PATH = get_nc_tpl_path()

## Parameters

In [ ]:
#| params
fname_in = '../../_data/accdb/mors/csv'
fname_out = '../../_data/output/helcom.nc'

## Utils

In [ ]:
def load_helcom(src_dir, 
                smp_types=['SEA', 'SED', 'BIO']):
    "Load HELCOM data and return them as individual dataframe by sample type"
    dfs = {}
    lut_smp_type = {'SEA': 'seawater', 'SED': 'sediment', 'BIO':'biota'}
    for smp_type in smp_types:
        fname_meas = smp_type + '02.csv'
        fname_smp = smp_type + '01.csv'
        df = pd.merge(pd.read_csv(Path(src_dir)/fname_meas), # measurements
                      pd.read_csv(Path(src_dir)/fname_smp), # sample
                      on='KEY', how='left')
        dfs[lut_smp_type[smp_type]] = df
    return dfs

def rename_cols(cols):
    "Flatten multiindex columns"
    new_cols = []
    for outer, inner in cols:
        if not inner:
            new_cols.append(outer)
        else:
            if outer == 'unc':
                new_cols.append(inner + '_' + outer)
            if outer == 'value':
                new_cols.append(inner)
    return new_cols

## Callbacks & Transformer

In [ ]:
class Callback(): order = 0

In [ ]:
class Transformer():
    def __init__(self, dfs, cbs=None): 
        fc.store_attr()
        self.logs = []
        
    def callback(self):
        run_cbs(self.cbs, self)
        
    def __call__(self):
        self.callback()
        return self.dfs

In [ ]:
def run_cbs(cbs, tfm=None):
    for cb in sorted(cbs, key=attrgetter('order')):
        if cb.__doc__: tfm.logs.append(cb.__doc__)
        cb(tfm)

## Configs

In [ ]:
# CONFIGS = {
#     'global_attr': {}
# }

## Load tables

In [ ]:
dfs = load_helcom(fname_in)

In [ ]:
dfs['seawater'].head()

,KEY,NUCLIDE,METHOD,< VALUE_Bq/m³,VALUE_Bq/m³,ERROR%_m³,DATE_OF_ENTRY_x,COUNTRY,LABORATORY,SEQUENCE,...,LONGITUDE (ddmmmm),LONGITUDE (dddddd),TDEPTH,SDEPTH,SALIN,TTEMP,FILT,MORS_SUBBASIN,HELCOM_SUBBASIN,DATE_OF_ENTRY_y
0,WKRIL2012003,CS137,NaN,NaN,5.3,32.0,08/20/14 00:00:00,90,KRIL,2012003,...,29.20,29.3333,NaN,0.0,NaN,NaN,NaN,11,11,08/20/14 00:00:00
1,WKRIL2012004,CS137,NaN,NaN,19.9,20.0,08/20/14 00:00:00,90,KRIL,2012004,...,29.20,29.3333,NaN,29.0,NaN,NaN,NaN,11,11,08/20/14 00:00:00
2,WKRIL2012005,CS137,NaN,NaN,25.5,20.0,08/20/14 00:00:00,90,KRIL,2012005,...,23.09,23.1500,NaN,0.0,NaN,NaN,NaN,11,3,08/20/14 00:00:00
3,WKRIL2012006,CS137,NaN,NaN,17.0,29.0,08/20/14 00:00:00,90,KRIL,2012006,...,27.59,27.9833,NaN,0.0,NaN,NaN,NaN,11,11,08/20/14 00:00:00
4,WKRIL2012007,CS137,NaN,NaN,22.2,18.0,08/20/14 00:00:00,90,KRIL,2012007,...,27.59,27.9833,NaN,39.0,NaN,NaN,NaN,11,11,08/20/14 00:00:00


In [ ]:
dfs['biota'].head()

,KEY,NUCLIDE,METHOD,< VALUE_Bq/kg,VALUE_Bq/kg,BASIS,ERROR%,NUMBER,DATE_OF_ENTRY_x,COUNTRY,...,BIOTATYPE,TISSUE,NO,LENGTH,WEIGHT,DW%,LOI%,MORS_SUBBASIN,HELCOM_SUBBASIN,DATE_OF_ENTRY_y
0,BVTIG2012041,CS134,VTIG01,<,0.010140,W,NaN,NaN,02/27/14 00:00:00,6.0,...,F,5,16.0,45.7,948.0,18.453,92.9,2,16,02/27/14 00:00:00
1,BVTIG2012041,K40,VTIG01,,135.300000,W,3.57,NaN,02/27/14 00:00:00,6.0,...,F,5,16.0,45.7,948.0,18.453,92.9,2,16,02/27/14 00:00:00
2,BVTIG2012041,CO60,VTIG01,<,0.013980,W,NaN,NaN,02/27/14 00:00:00,6.0,...,F,5,16.0,45.7,948.0,18.453,92.9,2,16,02/27/14 00:00:00
3,BVTIG2012041,CS137,VTIG01,,4.338000,W,3.48,NaN,02/27/14 00:00:00,6.0,...,F,5,16.0,45.7,948.0,18.453,92.9,2,16,02/27/14 00:00:00
4,BVTIG2012040,CS134,VTIG01,<,0.009614,W,NaN,NaN,02/27/14 00:00:00,6.0,...,F,5,17.0,45.9,964.0,18.458,92.9,2,16,02/27/14 00:00:00


In [ ]:
dfs['biota'].columns

Index(['KEY', 'NUCLIDE', 'METHOD', '< VALUE_Bq/kg', 'VALUE_Bq/kg', 'BASIS',
       'ERROR%', 'NUMBER', 'DATE_OF_ENTRY_x', 'COUNTRY', 'LABORATORY',
       'SEQUENCE', 'DATE', 'YEAR', 'MONTH', 'DAY', 'STATION',
       'LATITUDE ddmmmm', 'LATITUDE dddddd', 'LONGITUDE ddmmmm',
       'LONGITUDE dddddd', 'SDEPTH', 'RUBIN', 'BIOTATYPE', 'TISSUE', 'NO',
       'LENGTH', 'WEIGHT', 'DW%', 'LOI%', 'MORS_SUBBASIN', 'HELCOM_SUBBASIN',
       'DATE_OF_ENTRY_y'],
      dtype='object')

In [ ]:
dfs['biota']['BIOTATYPE'].unique()

array(['F', 'B', 'P'], dtype=object)

In [ ]:
# 42 of them
dfs['biota']['RUBIN'].unique()

array(['GADU MOR', 'SPRA SPR', 'CLUP HAR', 'MERL MNG', 'LIMA LIM',
       'PLEU PLA', 'PLAT FLE', 'SADU ENT', 'ENGR ENC', 'ESOX LUC',
       'MACO BAL', 'FUCU VES', 'ZOAR VIV', 'OSME EPE', 'MYOX SCO',
       'GYMN CER', 'GAST ACU', 'SCOM SCO', 'MYTI EDU', 'CYPR CAR',
       'ABRA BRA', 'STIZ LUC', 'RUTI RUT', 'PERC FLU', 'MYA ARE',
       'CRAN CRA', 'PLANKTON', 'CARD EDU', 'ARCT ISL', 'CLAD GLO',
       'FURC LUM', 'ANGU ANG', 'FISHLARVAE', 'ENCH CIM', 'ASTE RUB',
       'RHODOPHY', 'LAMI SAC', 'PSET MAX', 'GADU MOR  ', 'POLY FUC',
       'STUC PEC', 'ZANN PALU'], dtype=object)

In [ ]:
df_rubin = pd.read_csv(Path(fname_in) / 'RUBIN_NAME.csv'); df_rubin

,RUBIN_ID,RUBIN,SCIENTIFIC NAME,ENGLISH NAME
0,11,ABRA BRA,ABRAMIS BRAMA,BREAM
1,12,ANGU ANG,ANGUILLA ANGUILLA,EEL
2,13,ARCT ISL,ARCTICA ISLANDICA,ISLAND CYPRINE
3,14,ASTE RUB,ASTERIAS RUBENS,COMMON STARFISH
4,15,CARD EDU,CARDIUM EDULE,COCKLE
5,40,CH HI;BA,CHARA BALTICA,BALTIC STONEWORT
6,2,CLAD GLO,CLADOPHORA GLOMERATA,GREEN ALGAE
7,5,CLUP HAR,CLUPEA HARENGUS,HERRING
8,16,CRAN CRA,CRANGON CRANGON,BROWN SHRIMP
9,17,CYPR CAR,CYPRINUS CARPIO,CARP


In [ ]:
sci_name = df_rubin[df_rubin['RUBIN'].isin(dfs['biota']['RUBIN'].unique())]['SCIENTIFIC NAME'].values

In [ ]:
for name, resp in zip(sci_name, resps):
    print(f'SciName in Helcom db: {name}')
    if resp != -1: 
        print(f'# of items: {len(resp)}')
    else:
        print('Unknown!!!')
    print(resp, '\n******************************\n', )

NameError: name 'resps' is not defined

In [ ]:
import requests

In [ ]:
# load 
df_species = pd.read_excel('../files/lut/dbo_species.xlsx');
df_species['species']

0                            Not available
1                      Aristeus antennatus
2                             Apostichopus
3       Saccharina japonica var. religiosa
4                       Siganus fuscescens
                       ...                
1606             Gadiculus argenteus thori
1607                      Phorcus lineatus
1608                      Penaeus vannamei
1609                         Raja montagui
1610                  Scophthalmus rhombus
Name: species, Length: 1611, dtype: object

In [ ]:
# for each species
# check aphiaID, how much
for species in df_species['species']:
    print(species)

Not available
Aristeus antennatus
Apostichopus
Saccharina japonica var. religiosa
Siganus fuscescens
Alpheus dentipes
Hexagrammos agrammus
Ditrema temminckii
Parapristipoma trilineatum
Scombrops boops
Pseudopleuronectes schrenki
Desmarestia ligulata
Saccharina japonica
Neodilsea yendoana
Costaria costata
Sargassum yezoense
Acanthephyra pelagica
Sargassum ringgoldianum
Acanthephyra quadrispinosa
Sargassum thunbergii
Sargassum patens
Asterias rubens
Sargassum miyabei
Homarus gammarus
Acanthephyra stylorostratis
Acanthocybium solandri
Acanthopagrus bifasciatus
Acanthophora muscoides
Acanthophora spicifera
Acanthurus triostegus
Actinopterygii
Adamussium colbecki
Ahnfeltiopsis densa
Alepes melanoptera
Ampharetidae
Anchoviella lepidentostole
Anguillidae
Aphroditidae
Arnoglossus
Aurigequula fasciata
Balaenoptera musculus
Balaenoptera physalus
Balistes
Beryciformes
Bryopsis maxima
Callinectes sp.
Callorhinus ursinus
Carassius auratus auratus
Carcharhinus sorrah
Caridae
Clupea harengus
Cathorop

In [ ]:
dfs['biota']['TISSUE'].unique()

array([ 5,  1, 41,  3, 51, 43, 42, 12, 10, 18, 52, 20,  8, 54, 53])

In [ ]:
dfs['sediment'].head()

,KEY,NUCLIDE,METHOD,< VALUE_Bq/kg,VALUE_Bq/kg,ERROR%_kg,< VALUE_Bq/m²,VALUE_Bq/m²,ERROR%_m²,DATE_OF_ENTRY_x,...,LOWSLI,AREA,SEDI,OXIC,DW%,LOI%,MORS_SUBBASIN,HELCOM_SUBBASIN,SUM_LINK,DATE_OF_ENTRY_y
0,SKRIL2012048,RA226,NaN,NaN,35.0,26.0,NaN,NaN,NaN,08/20/14 00:00:00,...,20.0,0.006,NaN,NaN,NaN,NaN,11.0,11.0,NaN,08/20/14 00:00:00
1,SKRIL2012049,RA226,NaN,NaN,36.0,22.0,NaN,NaN,NaN,08/20/14 00:00:00,...,27.0,0.006,NaN,NaN,NaN,NaN,11.0,11.0,NaN,08/20/14 00:00:00
2,SKRIL2012050,RA226,NaN,NaN,38.0,24.0,NaN,NaN,NaN,08/20/14 00:00:00,...,2.0,0.006,NaN,NaN,NaN,NaN,11.0,11.0,NaN,08/20/14 00:00:00
3,SKRIL2012051,RA226,NaN,NaN,36.0,25.0,NaN,NaN,NaN,08/20/14 00:00:00,...,4.0,0.006,NaN,NaN,NaN,NaN,11.0,11.0,NaN,08/20/14 00:00:00
4,SKRIL2012052,RA226,NaN,NaN,30.0,23.0,NaN,NaN,NaN,08/20/14 00:00:00,...,6.0,0.006,NaN,NaN,NaN,NaN,11.0,11.0,NaN,08/20/14 00:00:00


## Data transformation pipeline

### Normalize nuclide names

#### Lower & strip

In [ ]:
class LowerStripRdnNameCB(Callback):
    "Convert nuclide names to lowercase & strip any trailing space(s)"
    def __call__(self, tfm):
        for k in tfm.dfs.keys():
            tfm.dfs[k]['NUCLIDE'] = tfm.dfs[k]['NUCLIDE'].apply(lambda x: x.lower().strip())

In [ ]:
dfs = load_helcom(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB()])
tfm()['seawater']['NUCLIDE'].unique()

array(['cs137', 'sr90', 'h3', 'cs134', 'pu238', 'pu239240', 'am241',
       'cm242', 'cm244', 'tc99', 'k40', 'ru103', 'sr89', 'sb125', 'nb95',
       'ru106', 'zr95', 'ag110m', 'cm243244', 'ba140', 'ce144', 'u234',
       'u238', 'co60', 'pu239', 'pb210', 'po210', 'np237', 'pu240',
       'mn54'], dtype=object)

#### Remap to MARIS nuclide names 

In [ ]:
def get_unique_nuclides(dfs):
    "Get list of unique radionuclide types measured across samples"
    nuclides = []
    for k in dfs.keys():
        nuclides += dfs[k]['NUCLIDE'].unique().tolist()
    return nuclides

get_unique_nuclides(tfm.dfs)[:5]

['cs137', 'sr90', 'h3', 'cs134', 'pu238']

In [ ]:
# Check if these variable names consistent with MARIS CDL
has_valid_varname(get_unique_nuclides(tfm.dfs), NC_TPL_PATH)

"pu239240" variable name not found in MARIS CDL
"cm243244" variable name not found in MARIS CDL
"cs134137" variable name not found in MARIS CDL
"pu239240" variable name not found in MARIS CDL
"pu238240" variable name not found in MARIS CDL
"pu239240" variable name not found in MARIS CDL
"cs134137" variable name not found in MARIS CDL
"k-40" variable name not found in MARIS CDL
"cs138" variable name not found in MARIS CDL
"cs139" variable name not found in MARIS CDL
"cs140" variable name not found in MARIS CDL
"cs141" variable name not found in MARIS CDL
"cs142" variable name not found in MARIS CDL
"cs143" variable name not found in MARIS CDL
"cs144" variable name not found in MARIS CDL
"cs145" variable name not found in MARIS CDL
"cs146" variable name not found in MARIS CDL


False

In [ ]:
# Create lut
varnames_lut = {n: n for n in set(get_unique_nuclides(tfm.dfs))}

In [ ]:
# Renaming above mentioned nuclides accordingly
varnames_lut['k-40'] = 'k40'
varnames_lut['cm243244'] = 'cm243_244_tot'
varnames_lut['cs134137'] = 'cs134_137_tot'
varnames_lut['pu239240'] = 'pu239_240_tot'
varnames_lut['pu238240'] = 'pu238_240_tot'

In [ ]:
class RemapRdnNameCB(Callback):
    "Remap to MARIS radionuclide names"
    def __init__(self, lut): fc.store_attr()        
    def __call__(self, tfm):
        for k in tfm.dfs.keys():
            tfm.dfs[k]['NUCLIDE'].replace(self.lut, inplace=True)            

In [ ]:
dfs = load_helcom(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(varnames_lut)])

tfm()['biota']['NUCLIDE'].unique()

array(['cs134', 'k40', 'co60', 'cs137', 'sr90', 'ag108m', 'mn54', 'co58',
       'ag110m', 'zn65', 'sb125', 'pu239_240_tot', 'ru106', 'be7',
       'ce144', 'pb210', 'po210', 'sb124', 'sr89', 'zr95', 'te129m',
       'ru103', 'nb95', 'ce141', 'la140', 'i131', 'ba140', 'pu238',
       'u235', 'bi214', 'pb214', 'pb212', 'tl208', 'ac228', 'ra223',
       'eu155', 'ra226', 'gd153', 'sn113', 'fe59', 'tc99', 'co57',
       'sn117m', 'eu152', 'sc46', 'rb86', 'ra224', 'th232',
       'cs134_137_tot', 'am241', 'ra228', 'th228', 'cs138', 'cs139',
       'cs140', 'cs141', 'cs142', 'cs143', 'cs144', 'cs145', 'cs146'],
      dtype=object)

In [ ]:
has_valid_varname(get_unique_nuclides(tfm.dfs), NC_TPL_PATH);

"cs138" variable name not found in MARIS CDL
"cs139" variable name not found in MARIS CDL
"cs140" variable name not found in MARIS CDL
"cs141" variable name not found in MARIS CDL
"cs142" variable name not found in MARIS CDL
"cs143" variable name not found in MARIS CDL
"cs144" variable name not found in MARIS CDL
"cs145" variable name not found in MARIS CDL
"cs146" variable name not found in MARIS CDL


### Parse time

In [ ]:
class ParseTimeCB(Callback):
    def __call__(self, tfm):
        #format_time = lambda x: date2num(x, units=get_cfgs('units')['time'])
        for k in tfm.dfs.keys():
            # Parse
            tfm.dfs[k]['time'] = pd.to_datetime(tfm.dfs[k].DATE, infer_datetime_format=True)

In [ ]:
dfs = load_helcom(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(varnames_lut),
                            ParseTimeCB()])

tfm()['seawater']['time'][:5]

0   2012-05-23
1   2012-05-23
2   2012-06-17
3   2012-05-24
4   2012-05-24
Name: time, dtype: datetime64[ns]

### Normalize uncertainty units

In [ ]:
# Make measurement and uncertainty units consistent
def fix_units(df, meas_col, unc_col):
    return df.apply(lambda row: row[unc_col] * row[meas_col]/100, axis=1)    

# Columns of interest
coi_units_unc = [('seawater', 'VALUE_Bq/m³', 'ERROR%_m³'), 
                 ('biota', 'VALUE_Bq/kg', 'ERROR%'),
                 ('sediment', 'VALUE_Bq/kg', 'ERROR%_kg')]

In [ ]:
class NormalizeUncUnitCB(Callback):
    "Convert uncertainty from % to activity unit"
    def __init__(self, coi): fc.store_attr()
    
    def __call__(self, tfm):
        for grp, val, unc in self.coi: 
            tfm.dfs[grp][unc] = self.fix_units(tfm.dfs[grp], val, unc)
            
    def fix_units(self, df, meas_col, unc_col):
        return df.apply(lambda row: row[unc_col] * row[meas_col]/100, axis=1)    

In [ ]:
dfs = load_helcom(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(varnames_lut),
                            ParseTimeCB(),
                            NormalizeUncUnitCB(coi_units_unc)])

tfm()['seawater'][['VALUE_Bq/m³', 'ERROR%_m³']][:5]


,VALUE_Bq/m³,ERROR%_m³
0,5.3,1.696
1,19.9,3.980
2,25.5,5.100
3,17.0,4.930
4,22.2,3.996


### Rename columns

In [ ]:
# Define columns of interest by sample type
coi_grp = {'seawater': ['NUCLIDE', 'VALUE_Bq/m³', 'ERROR%_m³', 'time',
                        'TDEPTH', 'LATITUDE (dddddd)', 'LONGITUDE (dddddd)'],
           'sediment': ['NUCLIDE', 'VALUE_Bq/kg', 'ERROR%_kg', 'time',
                        'TDEPTH', 'LATITUDE (dddddd)', 'LONGITUDE (dddddd)', 
                        'SEDI'],
            'biota': ['NUCLIDE', 'VALUE_Bq/kg', 'ERROR%', 'time',
                        'SDEPTH', 'LATITUDE ddmmmm', 'LONGITUDE ddmmmm',
                        'BIOTATYPE', 'TISSUE']}

In [ ]:
# Define column names renaming rules
renaming_rules = {
    'NUCLIDE': 'nuclide',
    'VALUE_Bq/m³': 'value',
    'VALUE_Bq/kg': 'value',
    'ERROR%_m³': 'unc',
    'ERROR%_kg': 'unc',
    'ERROR%': 'unc',
    'TDEPTH': 'depth',
    'SDEPTH': 'depth',
    'LATITUDE (dddddd)':'lat',
    'LATITUDE ddmmmm': 'lat',
    'LONGITUDE (dddddd)':'lon',
    'LONGITUDE ddmmmm': 'lon',
    # group specific
    'BIOTATYPE': 'bio_group',
    'TISSUE': 'body_part',
    'SEDI': 'sed_type'
}

In [ ]:
class RenameColumnCB(Callback):
    def __init__(self, coi, renaming_rules): fc.store_attr()
    
    def __call__(self, tfm):
        for k in tfm.dfs.keys():
            # Select cols of interest
            tfm.dfs[k] = tfm.dfs[k].loc[:, self.coi[k]]
            
            # Rename cols
            tfm.dfs[k].rename(columns=self.renaming_rules, inplace=True)

In [ ]:
dfs = load_helcom(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(varnames_lut),
                            ParseTimeCB(),
                            NormalizeUncUnitCB(coi_units_unc),
                            RenameColumnCB(coi_grp, renaming_rules)])

tfm()['seawater'].head(5)

,nuclide,value,unc,time,depth,lat,lon
0,cs137,5.3,1.696,2012-05-23,NaN,60.0833,29.3333
1,cs137,19.9,3.980,2012-05-23,NaN,60.0833,29.3333
2,cs137,25.5,5.100,2012-06-17,NaN,59.4333,23.1500
3,cs137,17.0,4.930,2012-05-24,NaN,60.2500,27.9833
4,cs137,22.2,3.996,2012-05-24,NaN,60.2500,27.9833


### Reshape: long to wide

In [ ]:
class ReshapeLongToWide(Callback):
    def __init__(self): fc.store_attr()
    
    def __call__(self, tfm):
        for k in tfm.dfs.keys():
            cols = ['nuclide']
            vals = ['value', 'unc']
            idx = list(set(tfm.dfs[k].columns) - set(cols + vals)) # All others

            tfm.dfs[k] = tfm.dfs[k].pivot_table(index=idx, 
                                                columns=cols, 
                                                values=vals).reset_index()

            # Flatten cols name
            tfm.dfs[k].columns = rename_cols(tfm.dfs[k].columns)

            # Set index
            tfm.dfs[k].index.name = 'sample'

In [ ]:
dfs = load_helcom(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(varnames_lut),
                            ParseTimeCB(),
                            NormalizeUncUnitCB(coi_units_unc),
                            RenameColumnCB(coi_grp, renaming_rules),
                            ReshapeLongToWide()])

tfm()['seawater'].head(5)

,lon,time,depth,lat,ag110m_unc,am241_unc,ba140_unc,ce144_unc,cm242_unc,cm243_244_tot_unc,...,pu240,ru103,ru106,sb125,sr89,sr90,tc99,u234,u238,zr95
sample,,,,,,,,,,,,,,,,,,,,,
0,0.0,2015-04-21,12.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.759,NaN,NaN,NaN,NaN
1,0.0,2015-04-23,4.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6.758,NaN,NaN,NaN,NaN
2,0.0,2015-04-30,13.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,2015-05-19,81.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,2015-05-20,69.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,7.255,NaN,NaN,NaN,NaN


### Encode time (seconds since ...)

In [ ]:
class EncodeTimeCB(Callback):
    "Encode time as `int` representing seconds since xxx"
    def __call__(self, tfm):
        format_time = lambda x: date2num(x, units=get_cfgs('units')['time'])
        for k in tfm.dfs.keys():    
            tfm.dfs[k]['time'] = tfm.dfs[k]['time'].apply(format_time)
  

In [ ]:
dfs = load_helcom(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(varnames_lut),
                            ParseTimeCB(),
                            NormalizeUncUnitCB(coi_units_unc),
                            RenameColumnCB(coi_grp, renaming_rules),
                            ReshapeLongToWide(),
                            EncodeTimeCB()])

tfm()['seawater'].head(5)

,lon,time,depth,lat,ag110m_unc,am241_unc,ba140_unc,ce144_unc,cm242_unc,cm243_244_tot_unc,...,pu240,ru103,ru106,sb125,sr89,sr90,tc99,u234,u238,zr95
sample,,,,,,,,,,,,,,,,,,,,,
0,0.0,1429574400,12.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.759,NaN,NaN,NaN,NaN
1,0.0,1429747200,4.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6.758,NaN,NaN,NaN,NaN
2,0.0,1430352000,13.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,1431993600,81.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,1432080000,69.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,7.255,NaN,NaN,NaN,NaN


### Sanitize coordinates

In [ ]:
class SanitizeLonLatCB(Callback):
    "Drop row when both longitude & latitude equal 0"
    def __call__(self, tfm):
        tfm.dfs = {grp: (df[(df.lon != 0) & (df.lat != 0)]) 
                   for grp, df in tfm.dfs.items()}

In [ ]:
dfs = load_helcom(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(varnames_lut),
                            ParseTimeCB(),
                            NormalizeUncUnitCB(coi_units_unc),
                            RenameColumnCB(coi_grp, renaming_rules),
                            ReshapeLongToWide(),
                            EncodeTimeCB(),
                            SanitizeLonLatCB()])

tfm()['seawater'].head(5)

,lon,time,depth,lat,ag110m_unc,am241_unc,ba140_unc,ce144_unc,cm242_unc,cm243_244_tot_unc,...,pu240,ru103,ru106,sb125,sr89,sr90,tc99,u234,u238,zr95
sample,,,,,,,,,,,,,,,,,,,,,
10,9.6333,544838400,16.0,54.8433,NaN,0.002907,NaN,NaN,0.004369,NaN,...,NaN,NaN,NaN,NaN,NaN,18.2,NaN,NaN,NaN,NaN
11,9.6333,597628800,17.0,54.8400,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,9.6333,690854400,16.0,54.8417,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,9.6333,724464000,17.0,54.8417,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,9.6352,629596800,17.0,54.8425,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Visualizing transform steps
tfm.logs

['Convert nuclide names to lowercase & strip any trailing space(s)',
 'Remap to MARIS radionuclide names',
 'Convert uncertainty from % to activity unit',
 'Encode time as `int` representing seconds since xxx',
 'Drop row when both longitude & latitude equal 0']

## Encode to NetCDF

### Feed global attributes

In [ ]:
kw = ['oceanography', 'Earth Science > Oceans > Ocean Chemistry> Radionuclides',
      'Earth Science > Human Dimensions > Environmental Impacts > Nuclear Radiation Exposure',
      'Earth Science > Oceans > Ocean Chemistry > Ocean Tracers, Earth Science > Oceans > Marine Sediments',
      'Earth Science > Oceans > Ocean Chemistry, Earth Science > Oceans > Sea Ice > Isotopes',
      'Earth Science > Oceans > Water Quality > Ocean Contaminants',
      'Earth Science > Biological Classification > Animals/Vertebrates > Fish',
      'Earth Science > Biosphere > Ecosystems > Marine Ecosystems',
      'Earth Science > Biological Classification > Animals/Invertebrates > Mollusks',
      'Earth Science > Biological Classification > Animals/Invertebrates > Arthropods > Crustaceans',
      'Earth Science > Biological Classification > Plants > Macroalgae (Seaweeds)']

In [ ]:
attrs = global_attrs(dfs, zotero_id, 
                     xtras={'keyword': kw,
                            'history': ', '.join(tfm.logs)})

### Metadata

In [ ]:
# Populating NetCDF global attributes metadata
item_id = '26VMZZ2Q'
global_attrs = Metadata(dfs)
CONFIGS['global_attr'] = global_attrs.fill(ZoteroItem(item_id, get_cfgs('zotero')))
CONFIGS['global_attr']

{'id': '',
 'title': 'Environmental database - Helsinki Commission Monitoring of Radioactive Substances',
 'summary': 'MORS Environment database has been used to collate data resulting from monitoring of environmental radioactivity in the Baltic Sea based on HELCOM Recommendation 26/3.\n\nThe database is structured according to HELCOM Guidelines on Monitoring of Radioactive Substances (https://www.helcom.fi/wp-content/uploads/2019/08/Guidelines-for-Monitoring-of-Radioactive-Substances.pdf), which specifies reporting format, database structure, data types and obligatory parameters used for reporting data under Recommendation 26/3.\n\nThe database is updated and quality assured annually by HELCOM MORS EG.',
 'keywords': '',
 'keywords_vocabulary': 'GCMD Science Keywords',
 'keywords_vocabulary_url': 'https://gcmd.earthdata.nasa.gov/static/kms/',
 'record': '',
 'featureType': '',
 'cdm_data_type': '',
 'Conventions': 'CF-1.10 ACDD-1.3',
 'publisher_name': 'Paul MCGINNITY, Iolanda OSVATH,

In [ ]:
kw = ['oceanography', 'Earth Science > Oceans > Ocean Chemistry> Radionuclides',
      'Earth Science > Human Dimensions > Environmental Impacts > Nuclear Radiation Exposure',
      'Earth Science > Oceans > Ocean Chemistry > Ocean Tracers, Earth Science > Oceans > Marine Sediments',
      'Earth Science > Oceans > Ocean Chemistry, Earth Science > Oceans > Sea Ice > Isotopes',
      'Earth Science > Oceans > Water Quality > Ocean Contaminants',
      'Earth Science > Biological Classification > Animals/Vertebrates > Fish',
      'Earth Science > Biosphere > Ecosystems > Marine Ecosystems',
      'Earth Science > Biological Classification > Animals/Invertebrates > Mollusks',
      'Earth Science > Biological Classification > Animals/Invertebrates > Arthropods > Crustaceans',
      'Earth Science > Biological Classification > Plants > Macroalgae (Seaweeds)']

In [ ]:
CONFIGS['global_attr']['keywords'] = ', '.join(kw); CONFIGS

{'global_attr': {'id': '',
  'title': 'Environmental database - Helsinki Commission Monitoring of Radioactive Substances',
  'summary': 'MORS Environment database has been used to collate data resulting from monitoring of environmental radioactivity in the Baltic Sea based on HELCOM Recommendation 26/3.\n\nThe database is structured according to HELCOM Guidelines on Monitoring of Radioactive Substances (https://www.helcom.fi/wp-content/uploads/2019/08/Guidelines-for-Monitoring-of-Radioactive-Substances.pdf), which specifies reporting format, database structure, data types and obligatory parameters used for reporting data under Recommendation 26/3.\n\nThe database is updated and quality assured annually by HELCOM MORS EG.',
  'keywords': 'oceanography, Earth Science > Oceans > Ocean Chemistry> Radionuclides, Earth Science > Human Dimensions > Environmental Impacts > Nuclear Radiation Exposure, Earth Science > Oceans > Ocean Chemistry > Ocean Tracers, Earth Science > Oceans > Marine Sedi

### To NetCDF

In [ ]:
def units_fn(grp_name,
             rdn_name):
    lut = {'seawater': 'Bq/m³', 
           'sediment': 'Bq/kg', 
           'biota': 'Bq/kg'}
    return lut[grp_name]

In [ ]:
to_netcdf(dfs, NC_TPL_PATH, fname_out, CONFIGS, units_fn)

% of discarded data for grp seawater: 0.0
% of discarded data for grp sediment: 0.0
% of discarded data for grp biota: 7.010663611527276
